In [56]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Layer, MaxPooling2D, Flatten
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import cv2

In [57]:
def read_img(real_path, fake_path):
    # Đọc ảnh từ cả hai thư mục
    real_image_files = sorted([os.path.join(real_path, f) for f in os.listdir(real_path) if f.endswith('.png')])
    fake_image_files = sorted([os.path.join(fake_path, f) for f in os.listdir(fake_path) if f.endswith('.png')])

    # Gộp ảnh từ cả hai thư mục vào một danh sách duy nhất
    all_image_files = real_image_files + fake_image_files

    # Tạo nhãn tương ứng cho ảnh (0 cho real, 1 cho fake)
    all_labels = [0] * len(real_image_files) + [1] * len(fake_image_files)

    return all_image_files, all_labels



In [58]:
def load_and_preprocess_image(image_path, image_size):
    img = cv2.imread(image_path)
    img = cv2.resize(img, image_size)
    img = img / 255.0  
    return img

In [63]:
def img_to_batch(batch_size, all_image_files, all_labels, image_size):
    num_batch = len(all_image_files) // batch_size

    batches = []

    for i in range(num_batch):

        batch_files = all_image_files[i * batch_size:(i + 1) * batch_size]
        batch_labels = all_labels[i * batch_size:(i + 1) * batch_size]
        
        for image in batch_files:
            batch_images = load_and_preprocess_image(image, image_size)
            batch_images = np.stack(batch_images, axis=0)
        
        batch_labels = np.array(batch_labels)

        batches.append((batch_images, batch_labels))

    return batches

In [60]:
#ResNet50 model
def extract_features(image_size, batches):
    #using resnet50 with imagenet weight
    extract_model = ResNet50(
        weights = 'imagenet',
        include_top = False,
        input_shape = image_size
    )

    output = extract_model.output
    output = MaxPooling2D()(output)
    output = Flatten()(output)

    model = Model(inputs=extract_model.input, outputs=output)

    #extract_feature
    features = []
    labels = []

    for batch_images, batch_labels in batches:
        # Extract features using ResNet50
        batch_features = model.predict(batch_images)
        features.append(batch_features)
        labels.append(batch_labels)
    
    features = np.vstack(features)
    labels = np.hstack(labels)

    return features, labels

In [61]:
def combine(real_path, fake_path):
    all_image_files, all_labels = read_img(real_path, fake_path)
    batches = img_to_batch(batch_size=16, all_image_files=all_image_files, all_labels=all_labels, image_size=(64, 64))
    features, labels = extract_features(image_size=(64, 64, 3), batches=batches)
    return features, labels

In [62]:
features = combine(r'D:\Sum24\DPL\Datasets\Fake', r'D:\Sum24\DPL\Datasets\Real')

ValueError: Input 0 of layer "functional" is incompatible with the layer: expected shape=(None, 64, 64, 3), found shape=(32, 64, 3)